# ABOUT


Datascientest's Datascientist continuous bootcamp - cohorte Mars2022 -  AeroBOT project

**Tutor**

* Alban THUET

**Authors:**

* Hélène ASSIR
* Hichem HADJI  
* [Ioannis STASINOPOULOS](https://www.linkedin.com/in/ioannis-stasinopoulos/)

</br>

---
</br>

**Version History**

Version | Date       | Author(s)  | Modification
--------|----------- | ---------  | --------------------------
X.X     | XX/XX/2022 | A.B        | modif
1.2     | 26/07/2022 | I.S        | Add substitution of AM.  
1.1     | 22/07/2022 | I.S, H.A.  | Special treatment of narratives using RegEx's 

This notebook can be executed entirely. 

It

* mounts the GDrive of our AeroBot project @gmail account.

* loads the data from the `train_data_final.pkl` file (cf. `0_test_set_creator_DO_NOT_MODIFY_20220630.ipynb`) under a pandas DataFrame named `df`, which contains `97417` entries and 96 columns.
These data do not contain any more UAS-related entries.

* Processes Narratives according to RegEx's and saves the output into a new column. This column is saved at the end into a .pkl file for loading them into other code.

# IMPORT PACKAGES


settings for  full / patial Narrative display. Helene?


In [ ]:
#######################
# Import packages
#######################
import numpy as np
import seaborn as sns

#######################
# Pandas
#######################
import pandas as pd
# Set pandas settings to show all data when using .head(), .columns etc.
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.set_option("display.colheader_justify","left") # left-justify the print output of pandas

### Display full columnwidth
# Set pandas settings to display full text columns
#pd.options.display.max_colwidth = None
# Restore pandas settings to display standard colwidth
pd.reset_option('display.max_colwidth')

import itertools # Pour créer des iterateurs

######################
# PLOTTING
######################
import matplotlib.pyplot as plt
%matplotlib inline
# Define global plot parameters for better readability and consistency among plots
# A complete list of the rcParams keys can be retrieved via plt.rcParams.keys() function
plt.rcParams['axes.titlesize'] = 30
plt.rcParams['axes.labelsize'] = 23
plt.rcParams['xtick.labelsize'] = 23
plt.rcParams['ytick.labelsize'] = 23
plt.rc('legend', fontsize=23)    # legend fontsize

# BOKEH 
from bokeh.plotting import figure # Importation de la classe figure qui permet de créer un graphique bokeh.
from bokeh.io import  push_notebook, output_notebook, show
output_notebook() # permet d'afficher tous les futurs graphiques dans l'output d'une cellule jupyter. Si cette instruction n'est pas lancée, la figure s'affichera dans un nouvel onglet.
from bokeh.models import ColumnDataSource
from bokeh.transform import dodge
from bokeh.models.tools import HoverTool

#####################
# NLP 
#####################
import re # for Regular Expression handling
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet') # WordNet lemmatizer
nltk.download('omw-1.4') # necessary for WordNet lemmatizer
from nltk.tokenize import word_tokenize # Usual tokenizer
from nltk.tokenize import TweetTokenizer # Special tokenizer;  "we'll", "didn't", etc. are considered as one word
from sklearn.feature_extraction.text import CountVectorizer # Vectorization
from nltk.corpus import stopwords # Import stopwords from nltk.corpus

###############################
# ML preprocessing and models
###############################
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import classification_report, confusion_matrix

import pickle as pkl # Saving data externally

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# LOAD DATA

## Mount GDrive

In [ ]:
#@title
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive/')

#check your present working directory 
%pwd

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


'/content/drive/My Drive/data/transformed'

In [ ]:
#@title
# move to the transformed data location (you can create a deeper structure, if needed, e.g. to save a trained model):
%cd /content/drive/MyDrive/data/transformed/

/content/drive/MyDrive/data/transformed


In [ ]:
#@title
!ls # list the content of the pwd

#!ls "/content/drive/MyDrive/Data_Science/Formations/DataScienceTest/projet/AeroBot/" # list contect of a speficic folder

'Copy of Qualified abbreviations_20220718.xlsx.gsheet'
'Data Dictionnary.xlsx'
 Narrative_PP_stemmed_21072022_TRAIN.pkl
 Narrative_Raw_Stemmed_21072022_TRAIN.pkl
'Qualified abbreviations_20220707_test.csv'
'Qualified abbreviations_20220708.csv'
'Qualified abbreviations_20220718.csv'
'Qualified abbreviations_20220718_Google_sheet.gsheet'
 test_data_final.pkl
 train_data_final.pkl


## Load data from .pkl file


In [ ]:
#@title
# Load the TRAIN data (97417 entries)
# Do not touch the TEST data until the end of the project!
# or the curse of the greek gods will fall upon you!

%cd /content/drive/MyDrive/data/transformed/
with open("train_data_final.pkl", "rb") as f:
    loaded_data = pkl.load(f)

df = loaded_data[0]
print("\nA Dataframe with", len(df), "entries has been loaded")

/content/drive/MyDrive/data/transformed

A Dataframe with 97417 entries has been loaded


# NARRATIVE PREPROCESSING


## On narratives : ReGex (1-letter & Units)


### Verify GPU activation

In [ ]:
# Verify GPU is active on Google Colab
import tensorflow as tf 
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please change your hardware accelerator")

Default GPU Device:/device:GPU:0


### Perform substitutions 

#### All except 'IF' & only AM preceeded by num

/!\ takes time to execute!

In [ ]:
# Compile Regular Expressions
r_R_L_C = re.compile(r"""
                  (?i)                   # turns on the case-insensitive mode of RegEx
                  \d+R 
                  | 
                  \d+L                   # at least a number *before* 'R' or 'L' or 'C'
                  |
                  \d+C
                  """, re.VERBOSE)

r_Right_Left_Center = re.compile(r"""
                  (?i)                   # turns on the case-insensitive mode of RegEx
                  \d+ Right 
                  | 
                  \d+ Left               # at least a number *before*
                  |
                  \d+ Center
                  """, re.VERBOSE)

r_Celcius_neg = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  \s                            # Avoid finding an aircraft model type, e.g. 'MH-65C'
                  -                             # Our hypothesis: most temp. indications are negative
                                                # otherwise confustion with RWY indication '36C' (center)
                  \d+                           
                  C
                  \s                            
                """, re.VERBOSE)

r_Celcius_degrees = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  \d+
                  \s                           
                  degrees                       # appears mostly in plural form
                  \s* 
                  C
                  \s                            # without this, you will match '120 degree clearing turn' 
                """, re.VERBOSE)

r_Mach_1 = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  Mach
                  \s*
                  0*\.\d{1,}                    # decimal number *after* 'Mach'
                  """, re.VERBOSE)

r_Mach_2 = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  0*\.\d{1,}                    # decimal number *before* 'Mach'
                  \s*
                  Mach
                  """, re.VERBOSE)

r_gust = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  \d+
                  g
                  \d+
                  """, re.VERBOSE)

# 0000Z format
r_Z_1 = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  \s
                  \d{4,4}                       # exactly 4 digits preceeding Zulu time (UTC +0) 'Z'    
                  Z
                  \s                            
                  """, re.VERBOSE)

# 00:00Z format
r_Z_2 = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  \s
                  \d{2,2}                       # exactly 2 digits     
                  \:
                  \d{2,2}
                  Z
                  \s                            
                  """, re.VERBOSE)

r_KTS = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  \d+
                  KTS
                  |
                  \d+
                  KT                        
                """, re.VERBOSE)

r_alt_1 = re.compile(r"""
                    (?i)                         # turns on the case-insensitive mode of RegEx
                    FL\d{1,}     
                    """, re.VERBOSE)


r_alt_2 = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  \d{0,2}[\.\,\;]
                  \d*
                  FT
                  """, re.VERBOSE)

r_am_num = re.compile(r"""                      # RegEx for 'am' or 'AM' preceeded by numbers
                  (?i)                          # turns on the case-insensitive mode of RegEx                 
                  [0-1X\w]{1,2}
                  [\;\:\,\.]*
                  [0-59]{1,2}
                  am                
                  """, re.VERBOSE)

r_pm = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx                 
                  [0-1X\w]{1,2}
                  [\;\:\,\.]*
                  [0-59]{1,2}
                  pm                
                  """, re.VERBOSE)

r_dist_MI = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  \d                            # at least one digit, sticked to 'mi'
                  MI
                  """, re.VERBOSE)

r_dist_SM = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  \d                            # at least one digit, sticked to 'mi'
                  SM
                  """, re.VERBOSE)

r_dist_NM = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  \d                            # at least one digit, sticked to 'mi'
                  NM
                  """, re.VERBOSE)

r_MIN = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  \d+                           # at least one digit. Without '+', it maches only '0minute' from '20minutes'
                  MINUTES                        
                  |
                  \d+
                  MINUTE
                  |
                  \d+
                  MINS
                  | 
                  \d+
                  MIN
                  """, re.VERBOSE)

r_HR = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  \d+
                  HR                        
                  """, re.VERBOSE)

r_LBS = re.compile(r"""
                  (?i)                          # turns on the case-insensitive mode of RegEx
                  \d+
                  LBS                        
                  """, re.VERBOSE)

In [ ]:
# Define replacements to perform
# All the findings of the RegEx's in the list will be substituted by the corresponding key
# Note the precise use a spaces the substitute, to avoid problems during tokenization
my_dict = {
          ' <RUNWAY> ': [r_R_L_C, r_Right_Left_Center],
          ' <TEMP_C_NEG> ': [r_Celcius_neg],
          ' <TEMP_C_POS> ': [r_Celcius_degrees],
          ' Mach ': [r_Mach_1, r_Mach_2],
          ' <GUST> ': [r_gust],
          ' <TIME> ': [r_Z_1, r_Z_2],
          ' KT ': [r_KTS],
          ' FT ': [r_alt_1, r_alt_2],
          ' Ante_Meridiem ': [r_am_num],
          ' PM ': [r_pm],
          ' MI': [r_dist_MI],
          ' SM ': [r_dist_SM],
          ' NM ': [r_dist_NM],
          ' MIN': [r_MIN],           # NO space at the end of ' MIN'
          ' HR': [r_HR],             # NO space at the end of ' HR'
          ' LBS ': [r_LBS]
          }


In [ ]:
def substitute_RegEx(my_dict):
  """
  Inputs: a dictionnary with RegEx's and subsitutes
  Go through the narratives and replace the findings of the RegEx's by the substitutes 
  passed as input (keys of the dictionnary).
  Write the new version of the narrative after the replacements into a new column of df 
  entitled 'Narrative_RegEx_subst'.
  """
  
  # Time function execution
  import time
  start_time = time.time()
  print(7*'-', "Execution started, why don't you grab a coffee...", 7*'-', '\n')

  # Copy the narratives into a new column
  df['Narrative_RegEx_subst'] = df['Narrative']

  # Initialize counters
  repl_counter = 0
  progress = 0

  # Loop through the narratives using their index (here ACN number)
  for idx in df['Narrative_RegEx_subst'].index:
    
    # Loop through the keys of the dict
    for k in my_dict.keys():  
      
      # Loop through the list of RegEx's that correspond to that key
      for regex in my_dict[k]:
        new_term = k
        repl_result = re.subn(regex, new_term, df['Narrative_RegEx_subst'].loc[idx])
        # The re.subn() method returns the new version of the target string after the replacements 
        # The second element is the number of replacements it has made
    
        # New version of the narrative, after the replacements
        df['Narrative_RegEx_subst'][idx] = repl_result[0]

        # Increment the counter of remplacements by the number of replacements done in the narrative
        repl_counter = repl_counter + repl_result[1]
      
    # Report on the execution progress  
    progress += 1
    if progress % 1000 == 0:
      print(f"{progress} narratives processed; {repl_counter} replacements done so far \n")

  end_time = time.time()
  print(f"--- Executed in {np.round((end_time - start_time)/60,1)} minutes ---")
  print(f"{repl_counter} replacements in total")

  return None

In [ ]:
# Call the function
# /!\ takes about 50min. to execute!
substitute_RegEx(my_dict)

------- Execution started, why don't you grab a coffee... ------- 



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1000 narratives processed; 784 replacements done so far 

2000 narratives processed; 1527 replacements done so far 

3000 narratives processed; 2386 replacements done so far 

4000 narratives processed; 3199 replacements done so far 

5000 narratives processed; 3869 replacements done so far 

6000 narratives processed; 4726 replacements done so far 

7000 narratives processed; 5541 replacements done so far 

8000 narratives processed; 6318 replacements done so far 

9000 narratives processed; 7206 replacements done so far 

10000 narratives processed; 8079 replacements done so far 

11000 narratives processed; 8880 replacements done so far 

12000 narratives processed; 9686 replacements done so far 

13000 narratives processed; 10552 replacements done so far 

14000 narratives processed; 11317 replacements done so far 

15000 narratives processed; 12104 replacements done so far 

16000 narratives processed; 12946 replacements done so far 

17000 narratives processed; 13750 replacements

#### AM & IF special treatment

We have treated 'am' preceeded by numeric values above.
Here, we treat the uppercase form 'AM', only in the narratives >= 2009, which are written in lowercase, except some uppercase abbreviations, such as 'AM'.

The same holds true for 'IF', which means 'Intermediate Fix'

In [ ]:
r_AM_2009 = re.compile(r"""
                  [\s\.\,\;\:\/]
                  AM
                  [\s\.\,\;\:\/]
                """, re.VERBOSE)

# "Intermediate Fix" abbreviation
r_IF = re.compile(r"""
                  [\s\.\,\;\:\/]
                  IF
                  [\s\.\,\;\:\/]
                  """, re.VERBOSE)

# Define replacements to perform
# All the findings of the RegEx's in the list will be substituted by the corresponding key
# Note the precise use a spaces the substitute, to avoid problems during tokenization
my_dict_2009 = {
          ' Ante_Meridiem ': [r_AM_2009],
                  ' <IF> ' : [r_IF]  # it actually means 'Intermediate Fix'
          }

In [ ]:
def substitute_RegEx_AM_IF(my_dict):
  """
  DEALS SPECIFICALLY WITH 'AM' - 'Ante Meridiem' and 'IF' - 'Intermediate Fix'
  on narratives >= 2009 ONLY!

  Inputs: a dictionnary with RegEx's and subsitutes
  Go through the narratives and replace the findings of the RegEx's by the substitutes 
  passed as input (keys of the dictionnary).
  Write the new version of the narrative after the replacements into a new column of df 
  entitled 'Narrative_RegEx_subst'.
  """
  
  # Time function execution
  import time
  start_time = time.time()
  print(7*'-', "Execution started, why don't you grab a coffee...", 7*'-', '\n')

  # # Copy the narratives into a new column
  # df['Narrative_RegEx_subst'] = df['Narrative']

  # Initialize counters
  repl_counter = 0
  progress = 0

  # Loop through the narratives AFTER 2009, using their index (here ACN number)
  for idx in df[df['Year']>=2009]['Narrative_RegEx_subst'].index:
    
    # Loop through the keys of the dict
    for k in my_dict.keys():  
      
      # Loop through the list of RegEx's that correspond to that key
      for regex in my_dict[k]:
        new_term = k
        repl_result = re.subn(regex, new_term, df['Narrative_RegEx_subst'].loc[idx])
        # The re.subn() method returns the new version of the target string after the replacements 
        # The second element is the number of replacements it has made
    
        # New version of the narrative, after the replacements
        df['Narrative_RegEx_subst'][idx] = repl_result[0]

        # Increment the counter of remplacements by the number of replacements done in the narrative
        repl_counter = repl_counter + repl_result[1]
      
    # Report on the execution progress  
    progress += 1
    if progress % 1000 == 0:
      print(f"{progress} narratives processed; {repl_counter} replacements done so far \n")

  end_time = time.time()
  print(f"--- Executed in {np.round((end_time - start_time)/60,1)} minutes ---")
  print(f"{repl_counter} replacements in total")

  return None

In [ ]:
# Call the function a 2nd time, for 'AM'
# /!\ takes about 50min. to execute!
substitute_RegEx_AM_IF(my_dict_2009)

------- Execution started, why don't you grab a coffee... ------- 



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1000 narratives processed; 5 replacements done so far 

2000 narratives processed; 15 replacements done so far 

3000 narratives processed; 26 replacements done so far 

4000 narratives processed; 34 replacements done so far 

5000 narratives processed; 40 replacements done so far 

6000 narratives processed; 47 replacements done so far 

7000 narratives processed; 58 replacements done so far 

8000 narratives processed; 67 replacements done so far 

9000 narratives processed; 74 replacements done so far 

10000 narratives processed; 81 replacements done so far 

11000 narratives processed; 82 replacements done so far 

12000 narratives processed; 88 replacements done so far 

13000 narratives processed; 92 replacements done so far 

14000 narratives processed; 93 replacements done so far 

15000 narratives processed; 99 replacements done so far 

16000 narratives processed; 100 replacements done so far 

17000 narratives processed; 101 replacements done so far 

18000 narratives proce

### Save the output externally

In [ ]:
##########################################################
# WARNING!! 
# If you execute this cell, you will OVERWRITTE the data!
##########################################################

%cd /content/drive/MyDrive/data/transformed/

# save the df['Narrative_RegEx_subst'] externally to avoid having to perform the 
# substitutions again
with open("Narrative_RegEx_subst_21072022_TRAIN.pkl", "wb") as f:
    pkl.dump([df['Narrative_RegEx_subst']], f) # saves the variables into a list

/content/drive/MyDrive/data/transformed
